In [1]:
import  os, glob
import  random, csv
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pickle
import cv2
from PIL import Image
import adapt
from adapt.feature_based import DANN
from tensorflow import keras
from keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Reshape
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

# The path of the datasets, use dict format
dataset_path = {"base": "dataset/ccpd/splitted_plates_base", 
                "challenge":"dataset/ccpd/splitted_plates_challenge",
               "db":"dataset/ccpd/splitted_plates_db",
               "fn":"dataset/ccpd/splitted_plates_fn",
               "weather":"dataset/ccpd/splitted_plates_weather"}
save_check_pt = './checkpoints_DANN'

def load_csv(root, filename, name2label):
    # From csv file return images dir,labels list
    if not os.path.exists(os.path.join(root, filename)):
        images = []
        for name in name2label.keys(): 
            images += glob.glob(os.path.join(root, name, '*.png'))
            images += glob.glob(os.path.join(root, name, '*.jpg'))
            images += glob.glob(os.path.join(root, name, '*.jpeg'))
        #print(len(images), images)
        random.shuffle(images) # shuffle images
        with open(os.path.join(root, filename), mode='w', newline='') as f:
            writer = csv.writer(f)
            for img in images:  
                name = img.split(os.sep)[-2]
                label = name2label[name]
                writer.writerow([img, label])
            print('written into csv file:', filename)
    # read existed csv
    images, labels = [], []
    with open(os.path.join(root, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)
            images.append(img)
            labels.append(label) 
    # return img dir and label
    return images, labels

def load_ccpd(root, mode='train'):
    name2label = {}  # "sq...":0
    # iterate sub dir, sort, while keep mapping
    for name in sorted(os.listdir(os.path.join(root))):
        # skip non file folder
        if not os.path.isdir(os.path.join(root, name)):
            continue
        # give each category a number
        name2label[name] = len(name2label.keys())
    images, labels = load_csv(root, 'images.csv', name2label)
    if mode == 'train':  # 20%
        images = images[:int(0.2 * len(images))]
        labels = labels[:int(0.2 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # all
        images = images
        labels = labels
    return images, labels, name2label

# def readCcpdImg(images_dir):
#     X = []
#     for img_dir in images_dir:
#         img = cv2.imread(img_dir)
#         img = cv2.resize(img,(32,32))
#         X.append(img)
#     X = np.array(X)
#     return X

def readCcpdImg(images_dir):
    X = []
    for img_dir in images_dir:
        img = Image.open(img_dir)
        img = img.convert('L') # conver to grayscale images
        img = img.resize([16, 32])
        img_np = np.asarray(img)
        X.append(img_np.reshape([-1]))
    X = np.array(X)
    return X

def get_img_label(path, mode,num_classes=35):
    images_dir, labels, table = load_ccpd(dataset_path[path],mode=mode)
    images = readCcpdImg(images_dir)
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=num_classes)
    return images,labels

def get_encoder(input_shape=(512,)):
    model = Sequential()
    model.add(Dense(512, activation='relu',
                    input_shape=input_shape))
    model.add(Dense(256, activation="relu"))
    model.add(Dense(128, activation="sigmoid"))
    return model

def get_task(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu',input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(34, activation="softmax"))
    return model

def get_discriminator(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu',input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

base_images,base_labels = get_img_label('base', 'all', num_classes=34)
weather_images,weather_labels = get_img_label('weather', 'all', num_classes=34)
weather_images_t,weather_labels_t = get_img_label('weather', 'train', num_classes=34)
challenge_images,challenge_labels = get_img_label('challenge', 'all', num_classes=34)
challenge_images_t,challenge_labels_t = get_img_label('challenge', 'train', num_classes=34)
db_images,db_labels = get_img_label('db', 'all', num_classes=34)
db_images_t,db_labels_t = get_img_label('db', 'train', num_classes=34)
fn_images,fn_labels = get_img_label('fn', 'all', num_classes=34)
fn_images_t,fn_labels_t = get_img_label('fn', 'train', num_classes=34)
print("load data finished!")

load data finished!


In [8]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(weather_images_t))
print(np.shape(weather_labels_t))
weather_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
weather_model.fit(base_images, base_labels,weather_images_t,weather_labels_t, epochs=80,verbose=1,batch_size = 32)
weather_model.score(base_images, base_labels)
weather_model.score(weather_images, weather_labels)
weather_model.save_weights(save_check_pt+ '/weather')

(8446, 512)
(8446, 34)
(762, 512)
(762, 34)
Epoch 1/80
286/286 [==============================] - 2s 4ms/step - loss: 2.5840 - acc: 0.3446 - disc_loss: 1.3876 - disc_acc: 0.5387
Epoch 2/80
286/286 [==============================] - 1s 4ms/step - loss: 1.7942 - acc: 0.5757 - disc_loss: 1.3503 - disc_acc: 0.5797
Epoch 3/80
286/286 [==============================] - 1s 4ms/step - loss: 1.3818 - acc: 0.6590 - disc_loss: 1.3541 - disc_acc: 0.5729
Epoch 4/80
286/286 [==============================] - 1s 4ms/step - loss: 1.1379 - acc: 0.6982 - disc_loss: 1.3481 - disc_acc: 0.5766
Epoch 5/80
286/286 [==============================] - 1s 4ms/step - loss: 0.9356 - acc: 0.7500 - disc_loss: 1.3375 - disc_acc: 0.5905
Epoch 6/80
286/286 [==============================] - 1s 4ms/step - loss: 0.8115 - acc: 0.7907 - disc_loss: 1.3204 - disc_acc: 0.6009
Epoch 7/80
286/286 [==============================] - 1s 4ms/step - loss: 0.7039 - acc: 0.8198 - disc_loss: 1.3146 - disc_acc: 0.6064
Epoch 8/80
286/286

286/286 [==============================] - 1s 4ms/step - loss: 0.0572 - acc: 0.9836 - disc_loss: 1.3194 - disc_acc: 0.5947
Epoch 62/80
286/286 [==============================] - 1s 4ms/step - loss: 0.0448 - acc: 0.9892 - disc_loss: 1.3250 - disc_acc: 0.5914
Epoch 63/80
286/286 [==============================] - 1s 4ms/step - loss: 0.0604 - acc: 0.9837 - disc_loss: 1.3201 - disc_acc: 0.5987
Epoch 64/80
286/286 [==============================] - 1s 4ms/step - loss: 0.0381 - acc: 0.9918 - disc_loss: 1.3187 - disc_acc: 0.5976
Epoch 65/80
286/286 [==============================] - 1s 4ms/step - loss: 0.0325 - acc: 0.9937 - disc_loss: 1.3329 - disc_acc: 0.5873
Epoch 66/80
286/286 [==============================] - 1s 4ms/step - loss: 0.0630 - acc: 0.9818 - disc_loss: 1.3259 - disc_acc: 0.5913
Epoch 67/80
286/286 [==============================] - 1s 4ms/step - loss: 0.0500 - acc: 0.9868 - disc_loss: 1.3258 - disc_acc: 0.5979
Epoch 68/80
286/286 [==============================] - 1s 4ms/step 

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(challenge_images_t))
print(np.shape(challenge_labels_t))
challenge_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
challenge_model.fit(base_images, base_labels,challenge_images_t,challenge_labels_t, epochs=88,verbose=1,batch_size = 32)
challenge_model.score(base_images, base_labels)
challenge_model.score(challenge_images, challenge_labels)
challenge_model.save_weights(save_check_pt+ '/challenge')

(8446, 512)
(8446, 34)
(109, 512)
(109, 34)
Epoch 1/88
266/266 [==============================] - 2s 4ms/step - loss: 2.5871 - acc: 0.3432 - disc_loss: 1.3922 - disc_acc: 0.5374
Epoch 2/88
266/266 [==============================] - 1s 4ms/step - loss: 1.9034 - acc: 0.5450 - disc_loss: 1.3527 - disc_acc: 0.5742
Epoch 3/88
266/266 [==============================] - 1s 3ms/step - loss: 1.4989 - acc: 0.6328 - disc_loss: 1.3287 - disc_acc: 0.5916
Epoch 4/88
266/266 [==============================] - 1s 3ms/step - loss: 1.2621 - acc: 0.6747 - disc_loss: 1.3244 - disc_acc: 0.6020
Epoch 5/88
266/266 [==============================] - 1s 4ms/step - loss: 1.0692 - acc: 0.7145 - disc_loss: 1.2905 - disc_acc: 0.6278
Epoch 6/88
266/266 [==============================] - 1s 4ms/step - loss: 0.9586 - acc: 0.7421 - disc_loss: 1.2952 - disc_acc: 0.6184
Epoch 7/88
266/266 [==============================] - 1s 4ms/step - loss: 0.8363 - acc: 0.7805 - disc_loss: 1.2829 - disc_acc: 0.6293
Epoch 8/88
266/266

266/266 [==============================] - 1s 4ms/step - loss: 0.0610 - acc: 0.9859 - disc_loss: 1.2670 - disc_acc: 0.6247
Epoch 62/88
266/266 [==============================] - 1s 4ms/step - loss: 0.0612 - acc: 0.9853 - disc_loss: 1.2718 - disc_acc: 0.6167
Epoch 63/88
266/266 [==============================] - 1s 4ms/step - loss: 0.0722 - acc: 0.9819 - disc_loss: 1.2609 - disc_acc: 0.6183
Epoch 64/88
266/266 [==============================] - 1s 4ms/step - loss: 0.0840 - acc: 0.9779 - disc_loss: 1.2631 - disc_acc: 0.6279
Epoch 65/88
266/266 [==============================] - 1s 4ms/step - loss: 0.0758 - acc: 0.9795 - disc_loss: 1.2273 - disc_acc: 0.6463
Epoch 66/88
266/266 [==============================] - 1s 4ms/step - loss: 0.0528 - acc: 0.9868 - disc_loss: 1.2486 - disc_acc: 0.6332
Epoch 67/88
266/266 [==============================] - 1s 4ms/step - loss: 0.0419 - acc: 0.9904 - disc_loss: 1.2470 - disc_acc: 0.6372
Epoch 68/88
266/266 [==============================] - 1s 4ms/step 

In [4]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(db_images_t))
print(np.shape(db_labels_t))
db_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
db_model.fit(base_images, base_labels,db_images_t,db_labels_t, epochs=88,verbose=1,batch_size = 32)
db_model.score(base_images, base_labels)
db_model.score(db_images, db_labels)
db_model.save_weights(save_check_pt+ '/db')

(8446, 512)
(8446, 34)
(373, 512)
(373, 34)
Epoch 1/88
269/269 [==============================] - 2s 4ms/step - loss: 2.6413 - acc: 0.3237 - disc_loss: 1.3721 - disc_acc: 0.5636
Epoch 2/88
269/269 [==============================] - 1s 4ms/step - loss: 1.9469 - acc: 0.5269 - disc_loss: 1.3487 - disc_acc: 0.5862
Epoch 3/88
269/269 [==============================] - 1s 4ms/step - loss: 1.5543 - acc: 0.6247 - disc_loss: 1.3456 - disc_acc: 0.5904
Epoch 4/88
269/269 [==============================] - 1s 4ms/step - loss: 1.2766 - acc: 0.6739 - disc_loss: 1.3146 - disc_acc: 0.6201
Epoch 5/88
269/269 [==============================] - 1s 4ms/step - loss: 1.1258 - acc: 0.7019 - disc_loss: 1.3198 - disc_acc: 0.6063
Epoch 6/88
269/269 [==============================] - 1s 4ms/step - loss: 0.9506 - acc: 0.7401 - disc_loss: 1.3342 - disc_acc: 0.5906
Epoch 7/88
269/269 [==============================] - 1s 4ms/step - loss: 0.8659 - acc: 0.7676 - disc_loss: 1.3082 - disc_acc: 0.6173
Epoch 8/88
269/269

269/269 [==============================] - 1s 4ms/step - loss: 0.0595 - acc: 0.9851 - disc_loss: 1.3100 - disc_acc: 0.6176
Epoch 62/88
269/269 [==============================] - 1s 4ms/step - loss: 0.0636 - acc: 0.9844 - disc_loss: 1.3167 - disc_acc: 0.6029
Epoch 63/88
269/269 [==============================] - 1s 4ms/step - loss: 0.0706 - acc: 0.9823 - disc_loss: 1.3185 - disc_acc: 0.6036
Epoch 64/88
269/269 [==============================] - 1s 4ms/step - loss: 0.0631 - acc: 0.9843 - disc_loss: 1.3242 - disc_acc: 0.5933
Epoch 65/88
269/269 [==============================] - 1s 4ms/step - loss: 0.0650 - acc: 0.9830 - disc_loss: 1.3089 - disc_acc: 0.6093
Epoch 66/88
269/269 [==============================] - 1s 4ms/step - loss: 0.0568 - acc: 0.9866 - disc_loss: 1.3126 - disc_acc: 0.6051
Epoch 67/88
269/269 [==============================] - 1s 4ms/step - loss: 0.0400 - acc: 0.9906 - disc_loss: 1.3194 - disc_acc: 0.5981
Epoch 68/88
269/269 [==============================] - 1s 4ms/step 

In [5]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(fn_images_t))
print(np.shape(fn_labels_t))
fn_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
fn_model.fit(base_images, base_labels,fn_images_t,fn_labels_t, epochs=70,verbose=1,batch_size = 32)
fn_model.score(base_images, base_labels)
fn_model.score(fn_images, fn_labels)
fn_model.save_weights(save_check_pt+ '/fn')

(8446, 512)
(8446, 34)
(48, 512)
(48, 34)
Epoch 1/70
264/264 [==============================] - 2s 4ms/step - loss: 2.6245 - acc: 0.3191 - disc_loss: 1.3955 - disc_acc: 0.5229
Epoch 2/70
264/264 [==============================] - 1s 4ms/step - loss: 1.8915 - acc: 0.5490 - disc_loss: 1.3484 - disc_acc: 0.5843
Epoch 3/70
264/264 [==============================] - 1s 4ms/step - loss: 1.4343 - acc: 0.6463 - disc_loss: 1.3150 - disc_acc: 0.6156
Epoch 4/70
264/264 [==============================] - 1s 4ms/step - loss: 1.1912 - acc: 0.6825 - disc_loss: 1.3160 - disc_acc: 0.6064
Epoch 5/70
264/264 [==============================] - 1s 4ms/step - loss: 1.0100 - acc: 0.7308 - disc_loss: 1.2703 - disc_acc: 0.6454
Epoch 6/70
264/264 [==============================] - 1s 4ms/step - loss: 0.8624 - acc: 0.7773 - disc_loss: 1.2486 - disc_acc: 0.6547
Epoch 7/70
264/264 [==============================] - 1s 4ms/step - loss: 0.7437 - acc: 0.8142 - disc_loss: 1.2333 - disc_acc: 0.6616
Epoch 8/70
264/264 [

264/264 [==============================] - 1s 4ms/step - loss: 0.0709 - acc: 0.9828 - disc_loss: 1.2002 - disc_acc: 0.6590
Epoch 62/70
264/264 [==============================] - 1s 4ms/step - loss: 0.0625 - acc: 0.9856 - disc_loss: 1.2163 - disc_acc: 0.6531
Epoch 63/70
264/264 [==============================] - 1s 4ms/step - loss: 0.0628 - acc: 0.9846 - disc_loss: 1.2118 - disc_acc: 0.6535
Epoch 64/70
264/264 [==============================] - 1s 4ms/step - loss: 0.0722 - acc: 0.9805 - disc_loss: 1.2020 - disc_acc: 0.6643
Epoch 65/70
264/264 [==============================] - 1s 4ms/step - loss: 0.0832 - acc: 0.9773 - disc_loss: 1.2103 - disc_acc: 0.6652
Epoch 66/70
264/264 [==============================] - 1s 4ms/step - loss: 0.0726 - acc: 0.9826 - disc_loss: 1.1898 - disc_acc: 0.6607
Epoch 67/70
264/264 [==============================] - 1s 4ms/step - loss: 0.0618 - acc: 0.9865 - disc_loss: 1.1856 - disc_acc: 0.6711
Epoch 68/70
264/264 [==============================] - 1s 4ms/step 